In [48]:
from __future__ import print_function
import pandas as pd
import numpy as np
import pickle
from pyproj import Proj, transform
import matplotlib.pyplot as plt
import scipy.constants
import math
from datetime import datetime

In [49]:
# import the gravity readings for each minute
# as processed in the GravMunging notebook
data = pd.read_pickle('AllGravityFinal.pkl')
data

,datetime,lat,long,gravity_mgals_grv,acx,acy,eoetvoes,free_air_anom,bouguer,water_depth_m,ground_speed_grv,heading
0,2016-03-23 00:00:00,-27.448170,153.094845,-2003.46,-0.0087,-0.0106,-2003.5,-468.3,-468.3,1200.0,0.0,31.8
1,2016-03-23 00:00:01,-27.448167,153.094833,-2003.46,-0.0092,-0.0099,-2003.5,-468.3,-468.3,NaN,0.0,47.0
2,2016-03-23 00:00:02,-27.448167,153.094833,-2003.46,-0.0092,-0.0099,-2003.5,-468.3,-468.3,NaN,0.0,46.0
3,2016-03-23 00:00:03,-27.448167,153.094833,-2003.46,-0.0092,-0.0099,-2003.5,-468.3,-468.3,NaN,0.0,46.0
4,2016-03-23 00:00:04,-27.448167,153.094833,-2003.46,-0.0092,-0.0099,-2003.5,-468.3,-468.3,NaN,0.0,36.8
5,2016-03-23 00:00:05,-27.448170,153.094845,-2003.46,-0.0092,-0.0099,-2003.5,-468.3,-468.3,1200.0,0.0,10.2
6,2016-03-23 00:00:06,-27.448167,153.094833,-2003.46,-0.0092,-0.0099,-2003.5,-468.3,-468.3,NaN,0.0,20.6
7,2016-03-23 00:00:07,-27.448167,153.094833,-2003.47,-0.0090,-0.0076,-2003.5,-468.3,-468.3,NaN,0.0,18.0
8,2016-03-23 00:00:08,-27.448167,153.094833,-2003.47,-0.0090,-0.0076,-2003.5,-468.3,-468.3,NaN,0.0,19.2
9,2016-03-23 00:00:09,-27.448167,153.094833,-2003.47,-0.0090,-0.0076,-2003.5,-468.3,-468.3,NaN,0.0,25.2


In [50]:
# downsample the data to 1minute intervals
subset = data.resample('T', closed='left',label='left', on='datetime').first()
subset.reset_index(drop=True, inplace=True)
subset

,datetime,lat,long,gravity_mgals_grv,acx,acy,eoetvoes,free_air_anom,bouguer,water_depth_m,ground_speed_grv,heading
0,2016-03-23 00:00:00,-27.448170,153.094845,-2003.46,-0.0087,-0.0106,-2003.5,-468.3,-468.3,1200.0,0.0,31.8
1,2016-03-23 00:01:00,-27.448167,153.094843,-2003.59,0.0051,0.0153,-2003.6,-468.4,-468.4,1200.0,0.0,340.7
2,2016-03-23 00:02:00,-27.448158,153.094847,-2003.79,0.0021,0.0076,-2003.8,-468.6,-468.6,1200.0,0.1,36.0
3,2016-03-23 00:03:00,-27.448142,153.094865,-2003.92,0.0185,0.0032,-2003.9,-468.8,-468.8,1200.0,0.2,10.6
4,2016-03-23 00:04:00,-27.448050,153.094820,-2004.13,-0.0119,0.0042,-2004.0,-468.9,-468.9,1200.0,0.4,318.6
5,2016-03-23 00:05:00,-27.447968,153.094713,-2004.12,-0.0162,-0.0002,-2004.1,-469.0,-469.0,1200.0,0.5,307.4
6,2016-03-23 00:06:00,-27.447883,153.094618,-2003.71,-0.0053,-0.0020,-2004.3,-469.1,-469.1,1200.0,0.4,328.9
7,2016-03-23 00:07:00,-27.447795,153.094577,-2003.05,0.0011,0.0048,-2004.3,-469.2,-469.2,1200.0,0.3,343.0
8,2016-03-23 00:08:00,-27.447710,153.094543,-2002.61,0.0039,0.0026,-2004.1,-469.0,-469.0,1200.0,0.3,337.9
9,2016-03-23 00:09:00,-27.447618,153.094503,-2002.54,0.0006,0.0003,-2003.9,-468.7,-468.7,1200.0,0.3,344.1


In [51]:
# import the start and end point details for each seismic line
# as processed in the LineEnds notebook
lines = pd.read_csv('SeismicLineDetails.csv')
lines

,line_no,line_name,UKOOA,shots_per_line,line_length_km,line_bearing_deg,line_spacing_m,f_shot_desc,f_shot_num,f_datetime,f_lat_dd,f_long_dd,f_depth_m,l_shot_desc,l_shot_num,l_datetime,l_lat_dd,l_long_dd,l_depth_m
0,1.0,EWobs_0,EWobs_0.0.p190\r\nEWobs_0.0.p291,2766,553.00,91.223,200.0,FGSP,989,2016-04-03 07:37:05,-27.259653,155.738196,4782.0,LGSP,12049,2016-04-06 04:17:21,-27.384354,161.324767,1717.0
1,2.0,EWobs_1,EWobs_1.0.p190\r\nEWobs_1.0.p291,798,159.40,91.223,200.0,FGSP,11341,2016-04-07 00:46:01,-27.383079,160.966945,1877.0,LGSP,14529,2016-04-07 19:40:50,-27.381816,162.577516,1360.0
2,3.0,EWobs_2,EWobs_2.0.p190\r\nEWobs_2.0.p291,402,80.20,271.223,200.0,FGSP,14311,2016-04-08 00:23:24,-27.382469,162.464014,1410.0,LGSP,12707,2016-04-08 10:47:29,-27.384835,161.653821,1537.0
3,4.0,D3A-Line2_0,D3A-Line2_0.0.p190\r\nD3A-Line2_0.0.p291,587,29.30,120.358,50.0,FGSP,936,2016-04-08 21:30:46,-27.318728,161.530517,1463.0,LGSP,1522,2016-04-09 00:47:40,-27.447439,161.789111,1527.0
4,5.0,D3A-Line7_0,D3A-Line7_0.0.p190\r\nD3A-Line7_0.0.p291,588,29.35,209.317,50.0,FGSP,1468,2016-04-09 06:28:43,-27.280966,161.727907,1474.0,LGSP,881,2016-04-09 09:54:03,-27.512145,161.583347,1568.0
5,6.0,D3A-Line4_0,D3A-Line4_0.0.p190\r\nD3A-Line4_0.0.p291,589,29.40,120.357,50.0,FGSP,934,2016-04-09 15:30:07,-27.342302,161.514610,1500.0,LGSP,1522,2016-04-09 18:39:01,-27.471469,161.774132,1575.0
6,7.0,D3A-Line5_0,D3A-Line5_0.0.p190\r\nD3A-Line5_0.0.p291,106,5.25,210.361,50.0,FGSP,1468,2016-04-10 00:40:14,-27.270951,161.707774,1442.0,LGSP,1363,2016-04-10 01:19:40,-27.312573,161.682499,1538.0
7,8.0,D3A-Line5_1,D3A-Line5_1.0.p190\r\nD3A-Line5_1.0.p291,586,29.25,30.361,50.0,FGSP,936,2016-04-10 07:29:42,-27.478053,161.578992,1593.0,LGSP,1521,2016-04-10 10:42:15,-27.247154,161.721979,1453.0
8,9.0,D1B-Line08_0,D1B-Line08_0.1.p190\r\nD1B-Line08_0.1.p291,629,31.40,18.875,50.0,FGSP,934,2016-04-10 23:35:08,-26.546036,160.934970,1599.0,LGSP,1562,2016-04-11 03:48:18,-26.276487,161.032104,1651.0
9,10.0,D1B-Line09_0,D1B-Line09_0.0.p190\r\nD1B-Line09_0.0.p291,253,12.60,260.538,50.0,FGSP,1371,2016-04-11 07:43:59,-26.384179,161.050762,1621.0,LGSP,1119,2016-04-11 09:14:37,-26.404665,160.926562,1669.0


In [52]:
# converting longitude decimal degrees to UTM
def makeX(row):
    longit = row['long']
    lat = row['lat']
    outProj = Proj("+proj=utm +zone=57J, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    inProj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    xy = transform(inProj,outProj,longit,lat)
    return xy[0]

# converting latiude decimal degrees to UTM
def makeY(row):
    longit = row['long']
    lat = row['lat']
    outProj = Proj("+proj=utm +zone=57J, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    inProj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    xy = transform(inProj,outProj,longit,lat)
    return xy[1]

# calculating the distance from the zero-point of each subsequent CMP
def calc_distance(row, x0, y0):
    return math.sqrt((row['x'] - x0) ** 2 + (row['y'] - y0) ** 2)

# create a dict, with the key being the seismic line name, and the value being a dataframe
# with each row representing a gravity reading from that seismic line
lineReadings = {}
for i in range(len(lines)):
    start = lines.loc[i,'f_datetime']
    end = lines.loc[i,'l_datetime']
    linename = lines.loc[i,'line_name']
    if linename == 'D3A-Line5_1':
        linename = 'D3A_Line5B'
    else:
        linename = linename.split('_')[0].replace('-','_')
    startres = subset['datetime'] >= start
    endres = subset['datetime'] <= end
    res = startres & endres
    lineData = subset[res].copy()
    lineData['line_name'] = linename
    # sort the data points, to ensure they are in one line from the first reading to the last
    # assume that the data are:
    # # in a straight line
    # # not directly north-south (ie no repeating X/long coordinates)
    lineData.sort_values('long', axis=0, inplace=True)
    # reindex the dataframe 
    lineData.reset_index(drop=True, inplace=True)
    # taking the CMP location at the zero-point (based on sort order)
    lineData['utm_zone'] = '57J'
    lineData['x'] = lineData.apply(lambda row: makeX(row), axis=1)
    lineData['y'] = lineData.apply(lambda row: makeY(row), axis=1)
    x0 = lineData['x'].iloc[0]
    y0 = lineData['y'].iloc[0]   
    lineData['length_along_line'] = lineData.apply(lambda row: calc_distance(row, x0, y0), axis = 1)
    lineReadings.update({linename:lineData.copy()})

In [53]:
lineReadings.keys()

['D3A_Line2',
 'D3A_Line3',
 'D3A_Line5B',
 'D3A_Line1',
 'D3A_Line6',
 'D3A_Line7',
 'D3A_Line4',
 'D3A_Line5',
 'D3A_Line8',
 'D1B_Line08',
 'D1B_Line09',
 'BB1B',
 'EWmcs',
 'EWobs',
 'D2A_Line2',
 'D1B_Line13',
 'D1B_Line12',
 'D1B_Line11',
 'D1B_Line10',
 'D1B_Line09B',
 'BV1B']

In [54]:
# d1b9 has two parts (interuption due to whales)
# need to join these together
# named to specifically line up with the seismic line name in DUG

# create a dataframe with the data merged
D1B_Line9m = pd.concat([lineReadings['D1B_Line09'],lineReadings['D1B_Line09B']],
                       join ='outer', axis=0, ignore_index=True)

# sort by x(long) values
D1B_Line9m.sort_values('long', axis=0, inplace=True)
# reindex the dataframe 
D1B_Line9m.reset_index(drop=True, inplace=True)
# update dataframe line name column
D1B_Line9m['line_name'] = 'D1B_Line9m'

# recalculate distance for merged dataframe
x0 = D1B_Line9m.loc[D1B_Line9m.index[0],'x']
y0 = D1B_Line9m.loc[D1B_Line9m.index[0],'y']
D1B_Line9m['length_along_line'] = D1B_Line9m.apply(lambda row: calc_distance(row, x0, y0), axis=1)
# remove the old lines from the dict
lineReadings.pop('D1B_Line09')
lineReadings.pop('D1B_Line09B')
# add the new line to the dict
lineReadings.update({'D1B_Line9m': D1B_Line9m})
lineReadings.keys()

['D3A_Line2',
 'D3A_Line3',
 'D3A_Line1',
 'D3A_Line6',
 'D3A_Line7',
 'D3A_Line4',
 'D3A_Line5',
 'D3A_Line8',
 'EWmcs',
 'D2A_Line2',
 'D1B_Line13',
 'D1B_Line12',
 'D1B_Line11',
 'D1B_Line10',
 'BV1B',
 'D3A_Line5B',
 'D1B_Line08',
 'D1B_Line9m',
 'BB1B',
 'EWobs']

In [55]:
D1B_Line9m

,datetime,lat,long,gravity_mgals_grv,acx,acy,eoetvoes,free_air_anom,bouguer,water_depth_m,ground_speed_grv,heading,line_name,utm_zone,x,y,length_along_line
0,2016-04-29 17:47:00,-26.431950,160.759912,-2067.89,0.0011,-0.0016,-2098.7,-489.4,-489.5,1744.0,4.8,259.7,D1B_Line9m,57J,675495.528213,7.075279e+06,0.000000
1,2016-04-29 17:46:00,-26.431713,160.761333,-2067.80,-0.0006,0.0005,-2098.6,-489.3,-489.3,1745.0,4.6,259.9,D1B_Line9m,57J,675637.680526,7.075303e+06,144.210668
2,2016-04-29 17:45:00,-26.431480,160.762778,-2067.89,-0.0016,0.0029,-2098.6,-489.3,-489.3,1746.0,4.7,261.5,D1B_Line9m,57J,675782.155641,7.075327e+06,290.644256
3,2016-04-29 17:44:00,-26.431273,160.764173,-2067.86,-0.0001,-0.0007,-2098.5,-489.1,-489.1,1743.0,4.5,256.4,D1B_Line9m,57J,675921.603991,7.075348e+06,431.649399
4,2016-04-29 17:43:00,-26.431062,160.765612,-2068.00,-0.0005,-0.0009,-2098.4,-489.1,-489.1,1743.0,4.8,261.1,D1B_Line9m,57J,676065.382496,7.075370e+06,577.015177
5,2016-04-29 17:42:00,-26.430845,160.767005,-2067.95,-0.0004,0.0005,-2098.4,-489.0,-489.0,1747.0,4.5,262.5,D1B_Line9m,57J,676204.680960,7.075392e+06,718.055103
6,2016-04-29 17:41:00,-26.430583,160.768412,-2067.78,-0.0018,-0.0026,-2098.1,-488.7,-488.7,1749.0,4.4,262.6,D1B_Line9m,57J,676345.378366,7.075419e+06,861.268587
7,2016-04-29 17:40:00,-26.430370,160.769815,-2068.14,0.0003,0.0021,-2098.3,-488.9,-488.9,1753.0,4.7,256.8,D1B_Line9m,57J,676485.670361,7.075440e+06,1003.224847
8,2016-04-29 17:39:00,-26.430127,160.771237,-2068.02,0.0001,-0.0010,-2098.2,-488.8,-488.8,1752.0,4.7,264.2,D1B_Line9m,57J,676627.837235,7.075465e+06,1147.564402
9,2016-04-29 17:38:00,-26.429910,160.772603,-2067.62,0.0012,0.0032,-2097.9,-488.5,-488.5,1745.0,4.7,258.6,D1B_Line9m,57J,676764.478394,7.075488e+06,1285.984913


In [56]:
# d3a5 has two parts (interuption due to whales)
# need to join these together
# named to specifically line up with the seismic line name in DUG after name fixing

# create a dataframe with the merged data
D3A_Line5 = pd.concat([lineReadings['D3A_Line5'],lineReadings['D3A_Line5B']],
                       join ='outer', axis=0, ignore_index=True)

# sort the values by x(long)
D3A_Line5.sort_values('long', axis=0, inplace=True)
# reindex the dataframe 
D3A_Line5.reset_index(drop=True, inplace=True)
# update dataframe line name column
D3A_Line5['line_name'] = 'D3A_Line5'

# recalc distances
x0 = D3A_Line5.loc[D3A_Line5.index[0],'x']
y0 = D3A_Line5.loc[D3A_Line5.index[0],'y']
D3A_Line5['length_along_line'] = D3A_Line5.apply(lambda row: calc_distance(row, x0, y0), axis=1)
# remove the old lines from the dict
lineReadings.pop('D3A_Line5')
lineReadings.pop('D3A_Line5B')
# add the new line to the dict
lineReadings.update({'D3A-Line5m': D3A_Line5})

# removing the '0' in the line number
lineReadings['D1B_Line8'] = lineReadings.pop('D1B_Line08')

lineReadings.keys()

['D3A_Line2',
 'D3A_Line3',
 'D3A_Line1',
 'D3A_Line6',
 'D3A_Line7',
 'D3A_Line4',
 'D3A_Line8',
 'D1B_Line8',
 'EWmcs',
 'D2A_Line2',
 'D1B_Line13',
 'D1B_Line12',
 'D1B_Line11',
 'D1B_Line10',
 'BV1B',
 'D1B_Line9m',
 'BB1B',
 'EWobs',
 'D3A-Line5m']

In [58]:
# combining it all into single dataframe for export as a .csv for inclusion with the LHR data release
lineNames = lineReadings.keys()
combined = lineReadings[lineNames[0]]
for i in range(1, len(lineNames)):
    combined = pd.concat([combined,lineReadings[lineNames[i]]], join ='outer', axis=0, ignore_index=True)
combined['line_name'].unique()

array(['D3A_Line2', 'D3A_Line3', 'D3A_Line1', 'D3A_Line6', 'D3A_Line7',
       'D3A_Line4', 'D3A_Line8', 'D1B_Line08', 'EWmcs', 'D2A_Line2',
       'D1B_Line13', 'D1B_Line12', 'D1B_Line11', 'D1B_Line10', 'BV1B',
       'D1B_Line9m', 'BB1B', 'EWobs', 'D3A_Line5'], dtype=object)

In [59]:
driftDetails = pd.read_csv('\\\\prod.lan\\active\\proj\\lhriodp\\work\\barnes\\grav_for_release\\driftcorrection.csv')
# driftDetails.drop(3, inplace = True)
driftDetails

for i in range(len(driftDetails)):
    s_year = int(driftDetails.loc[i,'start_date'][6::])
    s_month = int(driftDetails.loc[i,'start_date'][3:5])
    s_day = int(driftDetails.loc[i,'start_date'][0:2])
    s_hour = int(driftDetails.loc[i,'start_time'][0])
    s_min = int(driftDetails.loc[i,'start_time'][2:4])
    s_sec = int(driftDetails.loc[i,'start_time'][6:8])
    driftDetails.loc[i,'start_datetime'] = datetime(s_year, s_month, s_day, s_hour, s_min, s_sec)
    e_year = int(driftDetails.loc[i,'end_date'][6::])
    e_month = int(driftDetails.loc[i,'end_date'][3:5])
    e_day = int(driftDetails.loc[i,'end_date'][0:2])
    e_hour = int(driftDetails.loc[i,'end_time'][0])
    e_min = int(driftDetails.loc[i,'end_time'][2:4])
    e_sec = int(driftDetails.loc[i,'end_time'][6:8])
    driftDetails.loc[i,'end_datetime'] = datetime(e_year, e_month, e_day, e_hour, e_min, e_sec)

driftDetails.drop(['start_date', 'start_time', 'end_date', 'end_time'], axis = 1, inplace = True)
driftDetails

,Leg,drift_mgals,abs_grav_start_mgal,rel_grav_start_mgal,start_datetime,end_datetime
0,Leg 1,-0.06272,979150.6744,-2002.99,2016-03-22 03:42:02,2016-03-30 03:45:08
1,Leg 2,0.93832,979139.5971,-2014.13,2016-03-30 03:45:08,2016-04-20 04:01:05
2,Leg 3,2.13196,979152.0188,-2000.77,2016-04-20 04:01:05,2016-05-11 04:15:05


In [60]:
combined['ship_eoetvoes_correction_mgals'] = combined['eoetvoes'] - combined['gravity_mgals_grv']
combined['drift_correction_mgals'] = np.nan
combined['corrected_abs_grav_mgals'] = np.nan

for leg in range(len(driftDetails)):
    leg_start = driftDetails.loc[leg,'start_datetime']
    leg_end = driftDetails.loc[leg,'end_datetime']
    leg_length = leg_end - leg_start
    afterstart = combined['datetime'] >= leg_start
    beforeend = combined['datetime'] <= leg_end    
    inleg = afterstart & beforeend

    prop = (combined[inleg].loc[:,'datetime'] - leg_start) / leg_length
    combined.loc[inleg,'drift_correction_mgals'] = prop * driftDetails.loc[leg, 'drift_mgals']
    combined.loc[inleg,'corrected_abs_grav_mgals'] = (driftDetails.loc[leg,'abs_grav_start_mgal'] +
                       (combined[inleg]['gravity_mgals_grv'] - driftDetails.loc[leg,'rel_grav_start_mgal']) +
                      combined[inleg]['drift_correction_mgals'] +
                      combined[inleg]['ship_eoetvoes_correction_mgals'])
combined['line_name'].unique()

array(['D3A_Line2', 'D3A_Line3', 'D3A_Line1', 'D3A_Line6', 'D3A_Line7',
       'D3A_Line4', 'D3A_Line8', 'D1B_Line08', 'EWmcs', 'D2A_Line2',
       'D1B_Line13', 'D1B_Line12', 'D1B_Line11', 'D1B_Line10', 'BV1B',
       'D1B_Line9m', 'BB1B', 'EWobs', 'D3A_Line5'], dtype=object)

In [61]:
# rearranging the column order
fixed_col_order = ['datetime',
                     'line_name',
                   'lat',
                     'long',
                     'utm_zone',
                     'x',
                     'y',
                     'length_along_line',                   
                     'water_depth_m',
                     'ground_speed_grv',
                     'heading',                   
                   'gravity_mgals_grv',
                     'acx',
                     'acy',
                     'eoetvoes',
                     'free_air_anom',
                     'bouguer',
                     'ship_eoetvoes_correction_mgals',
                     'drift_correction_mgals',
                     'corrected_abs_grav_mgals']


combined = combined[fixed_col_order]


# changing the names of the columns to be more explict
fixed_names = ['datetime',
             'line_name',
             'latitude_dd',
             'longitude_dd',
             'utm_zone',
             'x',
             'y',
             'length_along_line_m',                   
             'water_depth_m',
             'ground_speed_knts',
             'heading_degT',                   
             'raw_gravity_mgals',
             'raw_acx_mgals',
             'raw_acy_mgals',
             'raw_eoetvoes_mgals',
             'raw_free_air_anom_mgals',
             'raw_bouguer_mgals',
             'ship_eoetvoes_correction_mgals',
             'drift_correction_mgals',
             'corrected_abs_grav_mgals']

combined.columns = fixed_names
combined

,datetime,line_name,latitude_dd,longitude_dd,utm_zone,x,y,length_along_line_m,water_depth_m,ground_speed_knts,heading_degT,raw_gravity_mgals,raw_acx_mgals,raw_acy_mgals,raw_eoetvoes_mgals,raw_free_air_anom_mgals,raw_bouguer_mgals,ship_eoetvoes_correction_mgals,drift_correction_mgals,corrected_abs_grav_mgals
0,2016-04-08 21:31:00,D3A_Line2,-27.318847,161.530770,57J,750426.232536,6.975709e+06,0.000000,1462.0,4.3,126.4,-2029.20,-0.0049,0.0030,-2003.7,-458.9,-458.9,25.50,0.434981,979150.462081
1,2016-04-08 21:32:00,D3A_Line2,-27.319405,161.531962,57J,750542.939955,6.975645e+06,133.234804,1462.0,4.2,111.2,-2029.22,0.0000,-0.0027,-2004.1,-459.4,-459.4,25.12,0.435012,979150.062112
2,2016-04-08 21:33:00,D3A_Line2,-27.319955,161.533195,57J,750663.789532,6.975582e+06,269.707244,1460.0,4.4,117.8,-2029.64,0.0009,0.0020,-2004.1,-459.4,-459.5,25.54,0.435043,979150.062143
3,2016-04-08 21:34:00,D3A_Line2,-27.320583,161.534475,57J,750789.081051,6.975509e+06,414.276741,1459.0,4.8,113.3,-2030.48,0.0020,-0.0012,-2004.0,-459.4,-459.4,26.48,0.435074,979150.162174
4,2016-04-08 21:35:00,D3A_Line2,-27.321235,161.535688,57J,750907.719403,6.975435e+06,554.277041,1458.0,4.7,119.0,-2031.16,-0.0005,-0.0026,-2004.3,-459.7,-459.7,26.86,0.435105,979149.862205
5,2016-04-08 21:36:00,D3A_Line2,-27.321943,161.536962,57J,751032.167920,6.975354e+06,702.595512,1459.0,4.9,114.7,-2031.12,0.0007,-0.0013,-2004.4,-459.9,-459.9,26.72,0.435136,979149.762236
6,2016-04-08 21:37:00,D3A_Line2,-27.322592,161.538220,57J,751155.265361,6.975279e+06,846.413415,1460.0,4.5,121.5,-2031.63,-0.0017,-0.0016,-2004.9,-460.4,-460.4,26.73,0.435167,979149.262267
7,2016-04-08 21:38:00,D3A_Line2,-27.323225,161.539517,57J,751282.189717,6.975207e+06,992.726094,1462.0,4.5,121.7,-2032.00,-0.0020,0.0031,-2004.7,-460.3,-460.3,27.30,0.435198,979149.462298
8,2016-04-08 21:39:00,D3A_Line2,-27.323855,161.540815,57J,751409.285156,6.975134e+06,1139.004562,1463.0,4.9,122.2,-2032.50,-0.0009,-0.0006,-2005.0,-460.6,-460.6,27.50,0.435229,979149.162329
9,2016-04-08 21:40:00,D3A_Line2,-27.324513,161.542113,57J,751536.315188,6.975059e+06,1286.815123,1465.0,4.5,118.2,-2033.34,0.0031,-0.0017,-2005.7,-461.3,-461.3,27.64,0.435260,979148.462360


In [62]:
combined['line_name'].unique()

array(['D3A_Line2', 'D3A_Line3', 'D3A_Line1', 'D3A_Line6', 'D3A_Line7',
       'D3A_Line4', 'D3A_Line8', 'D1B_Line08', 'EWmcs', 'D2A_Line2',
       'D1B_Line13', 'D1B_Line12', 'D1B_Line11', 'D1B_Line10', 'BV1B',
       'D1B_Line9m', 'BB1B', 'EWobs', 'D3A_Line5'], dtype=object)

In [63]:
# for exporting the data for integration with the forward models
filename = 'GravPerLine.pkl'
pickle.dump(combined, open(filename, 'wb'), protocol=2)
print('Data has been exported and saved as', filename)

Data has been exported and saved as GravPerLine.pkl


In [64]:
# # select which columns to include in the data release
cols_to_keep = ['datetime',
             'line_name',
             'latitude_dd',
             'longitude_dd',
             'utm_zone',
             'x',
             'y',
             'length_along_line_m',                   
             'water_depth_m',
             'ground_speed_knts',
             'heading_degT',                   
             'raw_gravity_mgals',
             'raw_acx_mgals',
             'raw_acy_mgals',
             'raw_eoetvoes_mgals',
             'drift_correction_mgals',
             'corrected_abs_grav_mgals']

final = combined[cols_to_keep]
final

,datetime,line_name,latitude_dd,longitude_dd,utm_zone,x,y,length_along_line_m,water_depth_m,ground_speed_knts,heading_degT,raw_gravity_mgals,raw_acx_mgals,raw_acy_mgals,raw_eoetvoes_mgals,drift_correction_mgals,corrected_abs_grav_mgals
0,2016-04-08 21:31:00,D3A_Line2,-27.318847,161.530770,57J,750426.232536,6.975709e+06,0.000000,1462.0,4.3,126.4,-2029.20,-0.0049,0.0030,-2003.7,0.434981,979150.462081
1,2016-04-08 21:32:00,D3A_Line2,-27.319405,161.531962,57J,750542.939955,6.975645e+06,133.234804,1462.0,4.2,111.2,-2029.22,0.0000,-0.0027,-2004.1,0.435012,979150.062112
2,2016-04-08 21:33:00,D3A_Line2,-27.319955,161.533195,57J,750663.789532,6.975582e+06,269.707244,1460.0,4.4,117.8,-2029.64,0.0009,0.0020,-2004.1,0.435043,979150.062143
3,2016-04-08 21:34:00,D3A_Line2,-27.320583,161.534475,57J,750789.081051,6.975509e+06,414.276741,1459.0,4.8,113.3,-2030.48,0.0020,-0.0012,-2004.0,0.435074,979150.162174
4,2016-04-08 21:35:00,D3A_Line2,-27.321235,161.535688,57J,750907.719403,6.975435e+06,554.277041,1458.0,4.7,119.0,-2031.16,-0.0005,-0.0026,-2004.3,0.435105,979149.862205
5,2016-04-08 21:36:00,D3A_Line2,-27.321943,161.536962,57J,751032.167920,6.975354e+06,702.595512,1459.0,4.9,114.7,-2031.12,0.0007,-0.0013,-2004.4,0.435136,979149.762236
6,2016-04-08 21:37:00,D3A_Line2,-27.322592,161.538220,57J,751155.265361,6.975279e+06,846.413415,1460.0,4.5,121.5,-2031.63,-0.0017,-0.0016,-2004.9,0.435167,979149.262267
7,2016-04-08 21:38:00,D3A_Line2,-27.323225,161.539517,57J,751282.189717,6.975207e+06,992.726094,1462.0,4.5,121.7,-2032.00,-0.0020,0.0031,-2004.7,0.435198,979149.462298
8,2016-04-08 21:39:00,D3A_Line2,-27.323855,161.540815,57J,751409.285156,6.975134e+06,1139.004562,1463.0,4.9,122.2,-2032.50,-0.0009,-0.0006,-2005.0,0.435229,979149.162329
9,2016-04-08 21:40:00,D3A_Line2,-27.324513,161.542113,57J,751536.315188,6.975059e+06,1286.815123,1465.0,4.5,118.2,-2033.34,0.0031,-0.0017,-2005.7,0.435260,979148.462360


In [65]:
# export the data as a .csv
final.to_csv('GA354_KR16-05_2DMSS_processed_gravity_fixed.csv')

In [12]:
# def calcBouguer(row):
#     density_contrast = 2310 - 1060
#     result = 2 * math.pi * scipy.constants.gravitational_constant * row['water_depth_m'] * density_contrast
#     return result * 100000 # converting from m/s2 to mGal
# combined['calc_bouguer'] = combined.apply(lambda row: calcBouguer(row), axis = 1)
    
# def calcEoetvoes(row):
#     Earth_angular_velocity = 0.000072921150                   # radians per SI second (mean solar second), from https://en.wikipedia.org/wiki/Earth%27s_rotation
#     Omega = Earth_angular_velocity 
#     phi = math.radians(row['lat'])                              # latitude in radians
#     knots_to_ms = 1 / 1.943846                                  # knots to m/s conversion factor, from http://pydoc.net/weather/0.9.1/weather.units.wind/
#     speed_ms = row['ground_speed_grv'] *  knots_to_ms           # knots to m/s
#     u = speed_ms * math.sin(math.radians(row['heading']))       # E-W velocity
#     v = speed_ms * math.cos(math.radians(row['heading']))       # N-S velocity
#     Re = 6378137                                                # radius of earth in m, from http://docs.astropy.org/en/stable/constants/
#     result = 2 * Omega * u * math.cos(phi) + (u**2 + v**2) / Re # Eoetvoes formula, from https://en.wikipedia.org/wiki/E%C3%B6tv%C3%B6s_effect
#     return result * 100000
# combined['calc_eoetvoes_correction'] = combined.apply(lambda row: calcEoetvoes(row), axis = 1)
# combined['calc_eoetvoes'] = combined['calc_eoetvoes_correction'] + combined['gravity_mgals_grv']

['datetime',
 'lat',
 'long',
 'gravity_mgals_grv',
 'acx',
 'acy',
 'eoetvoes',
 'free_air_anom',
 'bouguer',
 'water_depth_m',
 'ground_speed_grv',
 'heading',
 'line_name',
 'utm_zone',
 'x',
 'y',
 'length_along_line',
 'ship_eoetvoes_correction_mgals',
 'drift_correction_mgals',
 'corrected_abs_grav_mgals']